In [7]:
include("../LiPoSID.jl")

using LinearAlgebra
function ⊗(A::AbstractMatrix, B::AbstractMatrix)
    return kron(A,B)
end

function LME_operator_symbolic(H, J)
    
    Id = I(2)   
    U = -im*(Id ⊗ H - transpose(H) ⊗ Id)
    D = sum( 2*transpose(j')⊗j-Id⊗(j'*j) - transpose(j)*transpose(j')⊗Id for j in J )/2 
    
    Lᴸᴹᴱ = U + D
    
    return Lᴸᴹᴱ
end

function LME_operator_from_DMD(A)

    M = [ 0  0   1  1
          1  im  0  0 
          1  -im  0  0
          0  0  -1  1 ]/2
   
    Lᴰᴹᴰ =   M * A * inv(M)
   
    return Lᴰᴹᴰ 
   
end


LME_operator_from_DMD (generic function with 1 method)

In [156]:
using SymPy
@syms s

function Transfer_Function(A, b)
    inv(I(4)*s - A)*b
end

bₓ = [1, 0, 0, 1]

function SumCoeffs(poly)
    p = expand(numerator(poly))
    degrees = range(0,convert(Int32,degree(p, s)))
    obj = sum(p.coeff(s^i) for i in degrees)
    return obj
end

function sympy_to_dynamicpoly(sympy_expr::Sym)
    # Extract variables from the SymPy expression
    vars = free_symbols(sympy_expr)

    # Convert SymPy variables to string and then to Symbol
    var_symbols = [Symbol(string(v)) for v in vars]

    # Create DynamicPolynomials variables using @polyvar macro
    @eval begin
        @polyvar $(var_symbols...)
    end

    # Map SymPy variables to DynamicPolynomials variables
    var_map = Dict(vars[i] => eval(var_symbols[i]) for i in 1:length(vars))

    # Substitute using subs and convert to DynamicPolynomial
    dynamic_poly_expr = SymPy.subs(sympy_expr, var_map)
    dynamic_poly = eval(Meta.parse(string(dynamic_poly_expr)))
    return dynamic_poly
end

sympy_to_dynamicpoly (generic function with 1 method)

In [157]:
ω = symbols("ω", real=true)
γ = symbols("γ", real=true)

Hᴸᴹᴱ = [ ω        0
         0        0   ]


Jᴸᴹᴱ = [ 0     γ
         0     0. + 0im  ]

Lᴸᴹᴱ = LME_operator_symbolic(Hᴸᴹᴱ, [Jᴸᴹᴱ])

M = [   0  0   1  1
        1  im  0  0 
        1 -im  0  0
        0  0  -1  1  ] / 2

Aᴸᴹᴱ = real.(inv(M) * Lᴸᴹᴱ * M)

Gᴸᴹᴱ = Transfer_Function(Aᴸᴹᴱ, bₓ)

4-element Vector{Sym{PyCall.PyObject}}:
 (1.0*s + 0.5*γ^2)/(1.0*s^2 + 1.0*s*γ^2 + 0.25*γ^4 + 1.0*ω^2)
             1.0*ω/(1.0*s^2 + 1.0*s*γ^2 + 0.25*γ^4 + 1.0*ω^2)
                                1.0*γ^2/(1.0*s^2 + 1.0*s*γ^2)
                                                          1/s

In [158]:
data_dir = "../DATA/"
γ_list = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

γ_list_disp = [ "0.0795",  "0.2513", "0.7947", "2.5133", "7.9477", "25.133",  "79.477", "251.33"];

In [161]:
γᵢ = γ_list[1]
γᶠ = parse(Float64, γᵢ)

ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)

lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series

Aᴰᴹᴰ⁻ˢᴮ = LiPoSID.direct_DMD_01XY_b4_A([ρᵉ[1:lᵐᵃˣ], ρᵍ[1:lᵐᵃˣ], ρˣ[1:lᵐᵃˣ], ρʸ[1:lᵐᵃˣ]])

Lᴰᴹᴰ = LME_operator_from_DMD(Aᴰᴹᴰ⁻ˢᴮ)

M = [   0  0   1  1
        1  im  0  0 
        1 -im  0  0
        0  0  -1  1  ] / 2

@assert abs(LiPoSID.frobenius_norm2(inv(M) * Lᴰᴹᴰ * M - Aᴰᴹᴰ⁻ˢᴮ)) < 1e-9


Gᴸᴹᴱ = Transfer_Function(Aᴸᴹᴱ, bₓ)[1:3]

Gᴰᴹᴰ = Transfer_Function(Aᴰᴹᴰ⁻ˢᴮ, bₓ)[1:3]

ΔG = (together.(Gᴸᴹᴱ - Gᴰᴹᴰ)).^2

polys = numerator.(ΔG)

obj = sum(SumCoeffs.(polys))

objective = sympy_to_dynamicpoly(obj)

best_solution, best_method = LiPoSID.sos_min_newton(objective) 


*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
termination status: SLOW_PROGRESS
solution status: NO_SOLUTION
optimum = -6.230678402630443e-5
The local solver failed refining the solution!
No higher TS step of the TSSOS hierarchy!
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
termination status: INFEASIBLE
solution status: NO_SOLUTION
optimum = -5.070515409390805
The local solver failed refining the solution!
No higher TS step of the TSSOS hierarchy!
val_p = [-0.000333833246849315, -10695.62423503433]


(PolyVar{true}[γ, ω] => [19.00059973844902, 59.99187464592152], "scaled_tssos")

In [ ]:
ρᵍ₀ = [1. 0]' * [1 0]
ρᵉ₀ = [0. 1]' * [0 1]
ρˣ₀ = [1  1]' * [1 1]/2
ρʸ₀ = [1 -im]' * [1 -im]/2

max_negs = []
sum_negs = []
num_negs = []

for γᵢ in γ_list 

    ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
    ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
    ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
    ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)

    lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
    lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series

    Aᴰᴹᴰ⁻ˢᴮ = LiPoSID.direct_DMD_01XY_b4_A([ρᵉ[1:lᵐᵃˣ], ρᵍ[1:lᵐᵃˣ], ρˣ[1:lᵐᵃˣ], ρʸ[1:lᵐᵃˣ]])

end